In [1]:
modello_per_analisi = 'Opel_Corsa' 
comune_per_analisi = "Sant'Agata Bolognese"

### 0. Importazioni funzioni e librerie

In [2]:
import pandas as pd
import re
import requests
from geopy.distance import geodesic
import os
import sys
from pathlib import Path
import importlib


#from Utils.model_utils import estrai_modello, estrai_allestimento, estrai_cv, estrai_anni, unifica_allestimento
#from Utils.cleaning_functions import normalizza_testo, pulisci_cambio, pulisci_prezzo, pulisci_km, pulisci_cambio_dt_merged, pulisci_carburante, pulisci_indirizzo_AT
#from Utils.dataset_cleaning import clean_data_AS,clean_data_AT,clean_data_AR,clean_data_ASM, clean_data_SU
#from Utils.dataset_formatting import get_data_dummy, data_formatting

In [3]:
from pathlib import Path
import sys

project_root = Path.cwd().parents[0]

# Costruisco il path alla cartella Utils
prep_dir = project_root / '2_Data_Preparation' / 'Utils'

# Aggiungo il path ai moduli Python
if str(prep_dir) not in sys.path:
    sys.path.append(str(prep_dir))

# Ora posso importare i moduli come se fossi dentro Utils
from dataset_cleaning import clean_data_AS, clean_data_AT, clean_data_AR, clean_data_ASM, clean_data_SU
from dataset_formatting import get_data_dummy, data_formatting

✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/0_Località/distanza.csv (7978 righe, 3 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/0_Località/only_cap_per_comune.csv (7890 righe, 5 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/0_Località/only_comune_per_cap.csv (4657 righe, 5 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/0_Località/distanza.csv (7978 righe, 3 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/0_Località/only_cap_per_comune.csv (7890 righe, 5 colonne)
✅ File caricato: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/0_Località/only_comune_per_cap.csv (4657 righe, 5 colonne)


In [4]:
# Percorsi delle cartelle contenenti i moduli
config_path = Path.cwd().parents[1] / 'Data'/ 'config'
# Aggiungi al sys.path
sys.path.extend([str(config_path)])
print('Il config_dir risulta:', config_path, ',')
print("il cui contenuto della cartella:", os.listdir(config_path))
config_path
# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
    print(f"✅ Modulo di configurazione '{modulo_config}' caricato.")
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base
motorizzazioni_norm = config.motorizzazioni_norm
modelli_ord = config.modelli_ord
mappa_cv = config.mappa_cv
mappa_allestimenti = config.mappa_allestimenti

Il config_dir risulta: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/config ,
il cui contenuto della cartella: ['config_Audi_A3.py', 'config_Opel_Corsa.py', '__pycache__', 'config_BMW_Serie_1.py', 'config_Mercedes_Classe_A.py']
✅ Modulo di configurazione 'config_Opel_Corsa' caricato.


### 1. Dati

In [5]:
data_dir = Path.cwd().parents[1] / 'Data'

In [6]:
distanza = pd.read_csv(f'{data_dir}/Processed_data/0_Località/distanza.csv')
only_cap_per_comune = pd.read_csv(f'{data_dir}/Processed_data/0_Località/only_cap_per_comune.csv')
only_comune_per_cap = pd.read_csv(f'{data_dir}/Processed_data/0_Località/only_comune_per_cap.csv')


In [7]:
data_at = pd.read_csv(f'{data_dir}/Raw_data/{modello_per_analisi}/data_automobile_it.csv')
data_as = pd.read_csv(f'{data_dir}/Raw_data/{modello_per_analisi}/data_autoscout.csv')
data_asm = pd.read_csv(f'{data_dir}/Raw_data/{modello_per_analisi}/data_autosupermarket.csv')
data_su = pd.read_csv(f'{data_dir}/Raw_data/{modello_per_analisi}/data_subito_it.csv')


### 2. Pulizia singoli DF

#### 2.1. Autoscout

In [8]:
#Pre pulizia
print(data_as.info())
data_as.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Annuncio           2469 non-null   object 
 1   Link               2469 non-null   object 
 2   Marca              2469 non-null   object 
 3   Modello            2469 non-null   object 
 4   Modello_plus_info  2459 non-null   object 
 5   Prezzo             2386 non-null   float64
 6   Chilometraggio     2469 non-null   int64  
 7   Cambio             2469 non-null   object 
 8   Immatricolazione   2469 non-null   object 
 9   Carburante         2469 non-null   object 
 10  CV                 2469 non-null   object 
 11  Località           2469 non-null   object 
 12  Venditore          2469 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 250.9+ KB
None


,Annuncio,Link,Marca,Modello,Modello_plus_info,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,CV,Località,Venditore
0,Opel Corsa 1.5 Diesel Start/Stop Elegance PREZ...,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,1.5 Diesel Start/Stop Elegance PREZZO +IVA,4111.0,150000,Manuale,01/2021,Diesel,75 kW (102 CV),Elena • IT-38017 Mezzolombardo - Trento,Rivenditore
1,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,Corsa 1.2 Blitz Edition s,6200.0,6000,Manuale,03/2022,Benzina,55 kW (75 CV),Steven Mazzucco • IT-36016 Thiene,Rivenditore
2,Opel Corsa 1.5 GS Line s&s 100cv,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,1.5 GS Line s&s 100cv,7990.0,200000,Manuale,11/2020,Diesel,74 kW (101 CV),"Privato,IT-81039 villa literno",Privato
3,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,Corsa 1.2 Blitz Edition s,6200.0,6000,Manuale,03/2022,Benzina,55 kW (75 CV),Steven Mazzucco • IT-36016 Thiene,Rivenditore


In [9]:
data_AS = clean_data_AS(data_as, only_comune_per_cap, motorizzazioni_norm, modelli_ord, mappa_cv)
print(data_AS.info())
data_AS.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          2469 non-null   object 
 1   Link              2469 non-null   object 
 2   Modello           2304 non-null   object 
 3   Prezzo            2386 non-null   float64
 4   Chilometraggio    2469 non-null   Int64  
 5   Cambio            2469 non-null   object 
 6   Immatricolazione  2469 non-null   Int64  
 7   Carburante        2469 non-null   object 
 8   CV                2469 non-null   int64  
 9   Venditore         2469 non-null   object 
 10  CAP               2467 non-null   Int64  
 11  Comune            2428 non-null   object 
dtypes: Int64(3), float64(1), int64(1), object(7)
memory usage: 238.8+ KB
None


,Annuncio,Link,Modello,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,CV,Venditore,CAP,Comune
0,Opel Corsa 1.5 Diesel Start/Stop Elegance PREZ...,https://www.autoscout24.it/annunci/opel-corsa-...,1.5 Diesel,4111.0,150000,manuale,2021,Diesel,102,Rivenditore,38017,mezzolombardo
1,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,1.2 75cv,6200.0,6000,manuale,2022,Benzina,102,Rivenditore,36016,thiene
2,Opel Corsa 1.5 GS Line s&s 100cv,https://www.autoscout24.it/annunci/opel-corsa-...,1.5 Diesel,7990.0,200000,manuale,2020,Diesel,102,Privato,81039,villa literno
3,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,1.2 75cv,6200.0,6000,manuale,2022,Benzina,102,Rivenditore,36016,thiene
4,Opel Corsa 1.4 Turbo Start/Stop On,https://www.autoscout24.it/annunci/opel-corsa-...,None,7990.0,146000,manuale,2025,Benzina,102,Privato,6124,perugia
5,Opel Corsa Corsa 1.5 Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,1.5 Diesel,8000.0,118000,manuale,2020,Diesel,102,Privato,20100,NaN
6,Opel Corsa 1.5 GS Line s&s 100cv,https://www.autoscout24.it/annunci/opel-corsa-...,1.5 Diesel,7990.0,200000,manuale,2020,Diesel,102,Privato,81039,villa literno
7,Opel Corsa 1.4 Turbo Start/Stop On,https://www.autoscout24.it/annunci/opel-corsa-...,None,7990.0,146000,manuale,2025,Benzina,102,Privato,6124,perugia
8,Opel Corsa Corsa 1.2 s&s 75cv * SUPER PROMO,https://www.autoscout24.it/annunci/opel-corsa-...,1.2 75cv,7900.0,65000,manuale,2022,Benzina,102,Rivenditore,10040,almese
9,Opel Corsa Corsa 1.2 Edition s&s 75cv * SUPER ...,https://www.autoscout24.it/annunci/opel-corsa-...,1.2 75cv,7900.0,75000,manuale,2021,Benzina,102,Rivenditore,10040,almese


#### 2.2. Automobile.it

In [10]:
print(data_at.info())
data_at.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Annuncio            1686 non-null   object
 1   Link                1686 non-null   object
 2   Prezzo              1686 non-null   object
 3   Tipologia           1686 non-null   object
 4   Immatricolazione    1686 non-null   object
 5   Chilometraggio      1686 non-null   object
 6   Carburante          1686 non-null   object
 7   Cambio              1550 non-null   object
 8   Unico_Proprietario  360 non-null    object
 9   Località            1686 non-null   object
dtypes: object(10)
memory usage: 131.8+ KB
None


,Annuncio,Link,Prezzo,Tipologia,Immatricolazione,Chilometraggio,Carburante,Cambio,Unico_Proprietario,Località
0,Opel Corsa 1.2 Design & Tech,https://www.automobile.it/latiano-opel-corsa-1...,€ 11.699,Usato,Dicembre 2022,69.000 km,Benzina - Euro 6,Manuale,NaN,Latiano (BR)
1,Opel Corsa 1.2,https://www.automobile.it/san-cipriano-d-avers...,€ 9.500,Usato,Giugno 2020,103.000 km,Benzina - Euro 6,Manuale,NaN,San Cipriano d'Aversa (CE)
2,OPEL Corsa 1.2 100 CV Edition CAR PLAY,https://www.automobile.it/bernalda-opel-corsa-...,€ 12.899,Usato,Aprile 2022,43.300 km,Benzina,Manuale,NaN,Bernalda (MT)
3,OPEL Corsa 1.2 Elegance,https://www.automobile.it/savona-opel-corsa-1-...,€ 12.900,Usato,Luglio 2020,42.689 km,Benzina - Euro 6,Manuale,Unico proprietario,Savona (SV)


In [11]:
data_AT = clean_data_AT(data_at, only_cap_per_comune, motorizzazioni_norm, modelli_ord, mappa_cv)
print(data_AT.info())
data_AT.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          1686 non-null   object 
 1   Link              1686 non-null   object 
 2   Prezzo            1686 non-null   Int64  
 3   Immatricolazione  1550 non-null   Int64  
 4   Chilometraggio    1646 non-null   Int64  
 5   Carburante        1686 non-null   object 
 6   Cambio            1686 non-null   object 
 7   Modello           1616 non-null   object 
 8   CV                1616 non-null   float64
 9   Comune            1686 non-null   object 
 10  CAP               1684 non-null   float64
dtypes: Int64(3), float64(2), object(6)
memory usage: 150.0+ KB
None


,Annuncio,Link,Prezzo,Immatricolazione,Chilometraggio,Carburante,Cambio,Modello,CV,Comune,CAP
0,Opel Corsa 1.2 Design & Tech,https://www.automobile.it/latiano-opel-corsa-1...,11699,2022,69,Benzina,manuale,1.2 75cv,75.0,latiano,72022.0
1,Opel Corsa 1.2,https://www.automobile.it/san-cipriano-d-avers...,9500,2020,103,Benzina,manuale,1.2 75cv,75.0,san cipriano d aversa,81036.0
2,OPEL Corsa 1.2 100 CV Edition CAR PLAY,https://www.automobile.it/bernalda-opel-corsa-...,12899,2022,43,Benzina,manuale,1.2 100cv,100.0,bernalda,75012.0
3,OPEL Corsa 1.2 Elegance,https://www.automobile.it/savona-opel-corsa-1-...,12900,2020,42,Benzina,manuale,1.2 75cv,75.0,savona,17100.0


In [12]:
print(data_asm.info())
data_asm.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Annuncio          864 non-null    object
 1   Modello           864 non-null    object
 2   Link              864 non-null    object
 3   Prezzo            864 non-null    object
 4   Chilometraggio    864 non-null    object
 5   Cambio            864 non-null    object
 6   Immatricolazione  864 non-null    object
 7   Carburante        864 non-null    object
 8   Tipologia         864 non-null    object
 9   CV                864 non-null    object
 10  Località          864 non-null    object
dtypes: object(11)
memory usage: 74.4+ KB
None


,Annuncio,Modello,Link,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,Tipologia,CV,Località
0,Opel Corsa 1.2 Edition NEOPATENTATI PROMO!!,1.2 Edition NEOPATENTATI PROMO!!,https://autosupermarket.it/annunci/opel-corsa-...,8.900 €,69.000 km,Man. 5 marce,"Ott, 2022",Benzina,Usata,55kW (75CV),35042 Este (PD)
1,Opel Corsa 100 CV GS Line,100 CV GS Line,https://autosupermarket.it/annunci/opel-corsa-...,11.500 €,130.000 km,Man.,"Giu, 2020",Diesel,Usata,75kW (102CV),51018 Pieve a Nievole (PT)
2,Opel Corsa 1.2 Edition,1.2 Edition,https://autosupermarket.it/annunci/opel-corsa-...,13.200 €,38.800 km,Man. 5 marce,"Mar, 2021",Benzina,Usata,55kW (75CV),54100 Massa (MS)
3,Opel Corsa 1.2 75CV GS #VARI COLORI,1.2 75CV GS #VARI COLORI,https://autosupermarket.it/annunci/opel-corsa-...,15.400 €,15.000 km,Man. 6 marce,"Feb, 2024",Benzina,Usata,55kW (75CV),30030 Martellago (VE)


#### 2.3. Autosupermarket

In [13]:
data_ASM = clean_data_ASM(data_asm, motorizzazioni_norm, modelli_ord)
print(data_ASM.info())
data_ASM.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          864 non-null    object 
 1   Modello           841 non-null    object 
 2   Link              864 non-null    object 
 3   Prezzo            864 non-null    int64  
 4   Chilometraggio    864 non-null    Int64  
 5   Cambio            864 non-null    object 
 6   Immatricolazione  864 non-null    Int64  
 7   Carburante        864 non-null    object 
 8   CV                864 non-null    float64
 9   CAP               864 non-null    object 
 10  Comune            864 non-null    object 
dtypes: Int64(2), float64(1), int64(1), object(7)
memory usage: 76.1+ KB
None


,Annuncio,Modello,Link,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,CV,CAP,Comune
0,Opel Corsa 1.2 Edition NEOPATENTATI PROMO!!,1.2 75cv,https://autosupermarket.it/annunci/opel-corsa-...,8900,69,manuale,2022,Benzina,75.0,35042,este
1,Opel Corsa 100 CV GS Line,1.5 Diesel,https://autosupermarket.it/annunci/opel-corsa-...,11500,130,manuale,2020,Diesel,102.0,51018,pieve a nievole
2,Opel Corsa 1.2 Edition,1.2 75cv,https://autosupermarket.it/annunci/opel-corsa-...,13200,38,manuale,2021,Benzina,75.0,54100,massa
3,Opel Corsa 1.2 75CV GS #VARI COLORI,1.2 75cv,https://autosupermarket.it/annunci/opel-corsa-...,15400,15,manuale,2024,Benzina,75.0,30030,martellago


In [14]:
print(data_su.info())
data_su.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Annuncio          2069 non-null   object
 1   Prezzo            2069 non-null   object
 2   Tipologia         2069 non-null   object
 3   Immatricolazione  2069 non-null   object
 4   Chilometraggio    1901 non-null   object
 5   Carburante        2062 non-null   object
 6   Cambio            2037 non-null   object
 7   Classe Euro       1760 non-null   object
 8   Località          2069 non-null   object
 9   Provincia         1919 non-null   object
 10  Venditore         2069 non-null   object
 11  Link              2069 non-null   object
dtypes: object(12)
memory usage: 194.1+ KB
None


,Annuncio,Prezzo,Tipologia,Immatricolazione,Chilometraggio,Carburante,Cambio,Classe Euro,Località,Provincia,Venditore,Link
0,Opel Corsa 1.2 Elegance OK NEOPATENTATI,9.400 €,Usato,12/2021,135000 Km,Benzina,Manuale,Euro 6,Pistoia,PT,Privato,https://www.subito.it/auto/opel-corsa-1-2-eleg...
1,OPEL Corsa 6ª serie - Corsa 1.5 D 100 CV Edition,14.800 €,Usato,12/2022,57171 Km,Diesel,Manuale,Euro 6,Villafranca di Verona,VR,Privato,https://www.subito.it/auto/opel-corsa-6-serie-...
2,Opel Corsa VI 2023 1.2 GS s&s 75cv,13.500 €,Usato,03/2024,22658 Km,Benzina,Manuale,Euro 6,Erba,CO,Privato,https://www.subito.it/auto/opel-corsa-vi-2023-...
3,Opel corsa 1.2,14.800 €,Usato,03/2024,12000 Km,Benzina,Manuale,Euro 6,Viterbo,VT,Privato,https://www.subito.it/auto/opel-corsa-1-2-vite...


#### 2.4. Subito.it

In [15]:
data_SU = clean_data_SU(data_su, only_cap_per_comune, motorizzazioni_norm, modelli_ord, mappa_cv)
print(data_SU.info())
data_SU.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          2069 non-null   object 
 1   Prezzo            2069 non-null   int64  
 2   Immatricolazione  2069 non-null   int32  
 3   Chilometraggio    2069 non-null   Int64  
 4   Carburante        2069 non-null   object 
 5   Cambio            2069 non-null   object 
 6   Venditore         2069 non-null   object 
 7   Link              2069 non-null   object 
 8   Modello           1564 non-null   object 
 9   CV                1564 non-null   float64
 10  Comune            2069 non-null   object 
 11  CAP               2069 non-null   int64  
dtypes: Int64(1), float64(1), int32(1), int64(2), object(7)
memory usage: 188.0+ KB
None


/Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Source/2_Data_Preparation/Utils/dataset_cleaning.py:86: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_SU['Immatricolazione'] = pd.to_datetime(data_SU['Immatricolazione'], errors='coerce').dt.year


,Annuncio,Prezzo,Immatricolazione,Chilometraggio,Carburante,Cambio,Venditore,Link,Modello,CV,Comune,CAP
0,Opel Corsa 1.2 Elegance OK NEOPATENTATI,9400,2021,135000,Benzina,manuale,Privato,https://www.subito.it/auto/opel-corsa-1-2-eleg...,1.2 75cv,75.0,pistoia,51100
1,OPEL Corsa 6ª serie - Corsa 1.5 D 100 CV Edition,14800,2022,57171,Diesel,manuale,Privato,https://www.subito.it/auto/opel-corsa-6-serie-...,1.5 Diesel,100.0,villafranca verona,37069
2,Opel Corsa VI 2023 1.2 GS s&s 75cv,13500,2024,22658,Benzina,manuale,Privato,https://www.subito.it/auto/opel-corsa-vi-2023-...,1.2 75cv,75.0,erba,22036
3,Opel corsa 1.2,14800,2024,12000,Benzina,manuale,Privato,https://www.subito.it/auto/opel-corsa-1-2-vite...,1.2 75cv,75.0,viterbo,1100


### 3. Formattazione Dataset finale

In [16]:
data = pd.concat([data_AS, data_AT, data_ASM, data_SU
                  #data_AR
                  ], ignore_index=True)
# Output diagnostico per verificare l'unione
print("\n✅ Dataset unificato caricato correttamente.")
print(f"Numero di righe: {data.shape[0]}")
print(f"Numero di colonne: {data.shape[1]}")
print("\n📋 Informazioni dettagliate per il dataframe NON FORMATTATO:")
data.info()
print()


✅ Dataset unificato caricato correttamente.
Numero di righe: 7088
Numero di colonne: 12

📋 Informazioni dettagliate per il dataframe NON FORMATTATO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7088 entries, 0 to 7087
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          7088 non-null   object 
 1   Link              7088 non-null   object 
 2   Modello           6325 non-null   object 
 3   Prezzo            7005 non-null   Float64
 4   Chilometraggio    7048 non-null   Int64  
 5   Cambio            7088 non-null   object 
 6   Immatricolazione  6952 non-null   Int64  
 7   Carburante        7088 non-null   object 
 8   CV                6513 non-null   float64
 9   Venditore         4538 non-null   object 
 10  CAP               7084 non-null   object 
 11  Comune            7047 non-null   object 
dtypes: Float64(1), Int64(2), float64(1), object(8)
memory usage: 685.4+ KB



In [17]:
data['Venditore']=data['Venditore'].fillna("Rivenditore")

In [18]:
data.head()

,Annuncio,Link,Modello,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,CV,Venditore,CAP,Comune
0,Opel Corsa 1.5 Diesel Start/Stop Elegance PREZ...,https://www.autoscout24.it/annunci/opel-corsa-...,1.5 Diesel,4111.0,150000,manuale,2021,Diesel,102.0,Rivenditore,38017,mezzolombardo
1,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,1.2 75cv,6200.0,6000,manuale,2022,Benzina,102.0,Rivenditore,36016,thiene
2,Opel Corsa 1.5 GS Line s&s 100cv,https://www.autoscout24.it/annunci/opel-corsa-...,1.5 Diesel,7990.0,200000,manuale,2020,Diesel,102.0,Privato,81039,villa literno
3,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,1.2 75cv,6200.0,6000,manuale,2022,Benzina,102.0,Rivenditore,36016,thiene
4,Opel Corsa 1.4 Turbo Start/Stop On,https://www.autoscout24.it/annunci/opel-corsa-...,None,7990.0,146000,manuale,2025,Benzina,102.0,Privato,6124,perugia


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7088 entries, 0 to 7087
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Annuncio          7088 non-null   object 
 1   Link              7088 non-null   object 
 2   Modello           6325 non-null   object 
 3   Prezzo            7005 non-null   Float64
 4   Chilometraggio    7048 non-null   Int64  
 5   Cambio            7088 non-null   object 
 6   Immatricolazione  6952 non-null   Int64  
 7   Carburante        7088 non-null   object 
 8   CV                6513 non-null   float64
 9   Venditore         7088 non-null   object 
 10  CAP               7084 non-null   object 
 11  Comune            7047 non-null   object 
dtypes: Float64(1), Int64(2), float64(1), object(8)
memory usage: 685.4+ KB


In [20]:
data_formattato = data_formatting(data, only_cap_per_comune, distanza, comune_per_analisi, mappa_allestimenti, allestimento_performance, allestimento_sport, allestimento_middle, allestimento_base)
print("\n📋 Informazioni dettagliate per il dataframe FORMATTATO:")
data_formattato.info()


Ci sono dei valori doppi? True
Se si quanti ce ne sono? 661, su un totale di 7088
Valori doppi rimossi

📋 Informazioni dettagliate per il dataframe FORMATTATO:
<class 'pandas.core.frame.DataFrame'>
Index: 4821 entries, 0 to 6427
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Annuncio                4821 non-null   object
 1   Link                    4821 non-null   object
 2   Prezzo                  4821 non-null   int64 
 3   Anni                    4821 non-null   int64 
 4   Immatricolazione        4821 non-null   int64 
 5   Chilometraggio          4821 non-null   int64 
 6   Cambio                  4821 non-null   object
 7   Carburante              4821 non-null   object
 8   CV                      4821 non-null   int64 
 9   Allestimento            4821 non-null   object
 10  Allestimento_unificato  4821 non-null   object
 11  Modello                 4718 non-null   object
 12  Vend

In [21]:
data_formattato[data_formattato['Chilometraggio']>200000]

,Annuncio,Link,Prezzo,Anni,Immatricolazione,Chilometraggio,Cambio,Carburante,CV,Allestimento,Allestimento_unificato,Modello,Venditore,Distanza,Comune,CAP,Regione,Area
4942,Opel Corsa 1.5 D 100 CV Gs Line,https://www.subito.it/auto/opel-corsa-1-5-d-10...,8999,4,2021,205933,manuale,Diesel,100,GS,Sport,1.5 Diesel,Privato,484,giugliano campania,80014,Campania,Sud


In [22]:
# Visualizzazione dei valori unici delle colonne categoriali (escluse alcune colonne informative)
exclude_columns = ["località", "CAP", "Comune", "Annuncio", "Link", "Regione"]
object_columns = [
    col for col in data_formattato.select_dtypes(include='object').columns 
    if col not in exclude_columns]
for column in object_columns:
    print("\nValori unici delle variabili Categoriali, riferite ai veicoli:")
    print(f"Colonna: {column}: {data_formattato[column].unique()}")
    print("-" * 40)


Valori unici delle variabili Categoriali, riferite ai veicoli:
Colonna: Cambio: ['manuale' 'automatico']
----------------------------------------

Valori unici delle variabili Categoriali, riferite ai veicoli:
Colonna: Carburante: ['Diesel' 'Benzina' 'GPL' 'Ibrido' 'Elettrico']
----------------------------------------

Valori unici delle variabili Categoriali, riferite ai veicoli:
Colonna: Allestimento: ['Elegance' 'Edition' 'GS' 'Desing & Tech' 'Ultimate' '40 Anniversario']
----------------------------------------

Valori unici delle variabili Categoriali, riferite ai veicoli:
Colonna: Allestimento_unificato: ['Middle' 'Base' 'Sport' 'Altro']
----------------------------------------

Valori unici delle variabili Categoriali, riferite ai veicoli:
Colonna: Modello: ['1.5 Diesel' '1.2 75cv' None '1.2 100cv' '1.2 130cv']
----------------------------------------

Valori unici delle variabili Categoriali, riferite ai veicoli:
Colonna: Venditore: ['Rivenditore' 'Privato']
------------------

### 4. Dataset dummy

In [23]:
# --- 6. CREAZIONE DF DUMMY X ANALISI ---
data_dummy = get_data_dummy(data_formattato, allestimento_performance, allestimento_sport, allestimento_middle)
print("\n📋 Informazioni per il dataframe DUMMY:")
data_dummy.info()


📋 Informazioni per il dataframe DUMMY:
<class 'pandas.core.frame.DataFrame'>
Index: 4821 entries, 0 to 6427
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Prezzo                   4821 non-null   int64
 1   Anni                     4821 non-null   int64
 2   Chilometraggio           4821 non-null   int64
 3   Cambio                   4821 non-null   int64
 4   CV                       4821 non-null   int64
 5   Venditore                4821 non-null   int64
 6   Distanza                 4821 non-null   int64
 7   is_Diesel vs Benzina     4821 non-null   int64
 8   is_GPL vs Benzina        4821 non-null   int64
 9   is_Ibrido vs Benzina     4821 non-null   int64
 10  is_Elettrico vs Benzina  4821 non-null   int64
 11  is_sport vs base         4821 non-null   int64
 12  is_performance vs base   4821 non-null   int64
 13  is_middle vs base        4821 non-null   int64
 14  is NO vs NE          

### 5. Salvataggio

In [24]:
output_folder_df = data_dir/ 'Processed_data' / modello_per_analisi
sys.path.extend(str(output_folder_df))
output_folder_df.mkdir(parents=True, exist_ok=True)

print("Cartella di salvataggio:", output_folder_df)
data.to_csv(os.path.join(output_folder_df, f"data_NF_{modello_per_analisi}NF.csv"), index=False)
data_formattato.to_csv(os.path.join(output_folder_df, f"data_{modello_per_analisi}.csv"), index=False)
data_dummy.to_csv(os.path.join(output_folder_df, f"data_dummy_{modello_per_analisi}.csv"), index=False)
print("\n✅ Programma completato con successo! I dataset sono stati salvati nella cartella:\n", output_folder_df)

Cartella di salvataggio: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Opel_Corsa

✅ Programma completato con successo! I dataset sono stati salvati nella cartella:
 /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Data/Processed_data/Opel_Corsa
